   * https://blog.min.io/kafka_and_minio/: Publish from Kafka, Persist on MinIO
   * https://docs.confluent.io/kafka-connectors/s3-sink/current/overview.html#
      * Search "Quick Start"  

In [18]:
!kubectl -n kafka-main-cluster get kafkaconnector s3-sink-cdc -o yaml | yq

apiVersion: kafka.strimzi.io/v1beta2
kind: KafkaConnector
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"kafka.strimzi.io/v1beta2","kind":"KafkaConnector","metadata":{"annotations":{},"labels":{"strimzi.io/cluster":"my-connect"},"name":"s3-sink-cdc","namespace":"kafka-main-cluster"},"spec":{"class":"io.confluent.connect.s3.S3SinkConnector","config":{"consumer.security.protocol":"SSL","consumer.ssl.enabled.protocols":"TLSv1.2,TLSv1.1","consumer.ssl.key.password":"connector1234","consumer.ssl.keystore.location":"/var/private/ssl/kafka.sink.keystore.jks","consumer.ssl.keystore.password":"connector1234","consumer.ssl.truststore.type":"JKS","flush.size":1000,"format.class":"io.confluent.connect.s3.format.avro.AvroFormat","key.converter":"io.confluent.connect.avro.AvroConverter","key.converter.schema.registry.url":"http://main-registry-schema-registry.kafka-main-cluster.svc:8081","key.converter.schemas.enable":true,"partitioner.class":"i

In [19]:
!kubectl -n kafka-main-cluster get kafkaconnector s3-sink-cdc -o jsonpath='{.status.conditions[0].message}'

The following tasks have failed: 0, see connectorStatus for more details.

In [22]:
!kubectl -n kafka-main-cluster get kafkaconnector s3-sink-cdc -o jsonpath='{.status.connectorStatus.tasks[0].trace}'

org.apache.kafka.connect.errors.ConnectException: com.amazonaws.SdkClientException: Unable to execute HTTP request: PKIX path building failed: sun.security.provider.certpath.SunCertPathBuilderException: unable to find valid certification path to requested target
	at io.confluent.connect.s3.S3SinkTask.start(S3SinkTask.java:142)
	at org.apache.kafka.connect.runtime.WorkerSinkTask.initializeAndStart(WorkerSinkTask.java:329)
	at org.apache.kafka.connect.runtime.WorkerTask.doRun(WorkerTask.java:202)
	at org.apache.kafka.connect.runtime.WorkerTask.run(WorkerTask.java:259)
	at org.apache.kafka.connect.runtime.isolation.Plugins.lambda$withClassLoader$1(Plugins.java:236)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:539)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1136)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExe

In [6]:
!echo | openssl s_client -connect minio.minio-tenant-1.svc:443

Connecting to 10.107.199.169
CONNECTED(00000003)
depth=0 O=system:nodes, CN=system:node:*.minio-tenant-1-hl.minio-tenant-1.svc.cluster.local
verify error:num=20:unable to get local issuer certificate
verify return:1
depth=0 O=system:nodes, CN=system:node:*.minio-tenant-1-hl.minio-tenant-1.svc.cluster.local
verify error:num=21:unable to verify the first certificate
verify return:1
depth=0 O=system:nodes, CN=system:node:*.minio-tenant-1-hl.minio-tenant-1.svc.cluster.local
verify return:1
---
Certificate chain
 0 s:O=system:nodes, CN=system:node:*.minio-tenant-1-hl.minio-tenant-1.svc.cluster.local
   i:CN=minikubeCA
   a:PKEY: id-ecPublicKey, 256 (bit); sigalg: RSA-SHA256
   v:NotBefore: Feb 13 21:44:05 2024 GMT; NotAfter: Feb 12 21:44:05 2025 GMT
---
Server certificate
-----BEGIN CERTIFICATE-----
MIIDoDCCAoigAwIBAgIRAIhqoPw6Q4xQJDg4t076CwAwDQYJKoZIhvcNAQELBQAw
FTETMBEGA1UEAxMKbWluaWt1YmVDQTAeFw0yNDAyMTMyMTQ0MDVaFw0yNTAyMTIy
MTQ0MDVaMGIxFTATBgNVBAoTDHN5c3RlbTpub2RlczFJMEcGA1UEAwxAc3lzdGVt

In [7]:
!echo | openssl s_client  -connect minio.minio-tenant-1.svc:443 2> /dev/null \
      | sed -ne '/-BEGIN CERTIFICATE-/,/-END CERTIFICATE-/p'

-----BEGIN CERTIFICATE-----
MIIDoDCCAoigAwIBAgIRAIhqoPw6Q4xQJDg4t076CwAwDQYJKoZIhvcNAQELBQAw
FTETMBEGA1UEAxMKbWluaWt1YmVDQTAeFw0yNDAyMTMyMTQ0MDVaFw0yNTAyMTIy
MTQ0MDVaMGIxFTATBgNVBAoTDHN5c3RlbTpub2RlczFJMEcGA1UEAwxAc3lzdGVt
Om5vZGU6Ki5taW5pby10ZW5hbnQtMS1obC5taW5pby10ZW5hbnQtMS5zdmMuY2x1
c3Rlci5sb2NhbDBZMBMGByqGSM49AgEGCCqGSM49AwEHA0IABNyNfQ/qFKkC62Gw
ug5v0gkY/XzbZOAG7SjVvwAdZGCQkpY8IE0ZKLszL53qmU9hFo+AXVtObeAR26PH
ekx0MZ+jggFnMIIBYzAOBgNVHQ8BAf8EBAMCBaAwEwYDVR0lBAwwCgYIKwYBBQUH
AwEwDAYDVR0TAQH/BAIwADAfBgNVHSMEGDAWgBTc6/VhSXvBX4PJTBcJlMz6la1q
WTCCAQsGA1UdEQSCAQIwgf+CTm1pbmlvLXRlbmFudC0xLXNzLTAtezAuLi4yfS5t
aW5pby10ZW5hbnQtMS1obC5taW5pby10ZW5hbnQtMS5zdmMuY2x1c3Rlci5sb2Nh
bIIlbWluaW8ubWluaW8tdGVuYW50LTEuc3ZjLmNsdXN0ZXIyLnhwdIIUbWluaW8u
bWluaW8tdGVuYW50LTGCGG1pbmlvLm1pbmlvLXRlbmFudC0xLnN2Y4IzKi5taW5p
by10ZW5hbnQtMS1obC5taW5pby10ZW5hbnQtMS5zdmMuY2x1c3RlcjIueHB0giEq
Lm1pbmlvLXRlbmFudC0xLnN2Yy5jbHVzdGVyMi54cHQwDQYJKoZIhvcNAQELBQAD
ggEBAK84x9j+dMEc2XKZ5PeE20B5UMgdSSZTifpkHdDE6MOIomSZNE9ajtvMCO